#Step 5
Optimization with uncertain graph weights

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy

import Code.utils as utils
import Code.environment.settings as settings
from Code.environment.Environment import Environment
from Code.GreedyLearner import GreedyLearner
from Code.environment.Customer import Customer
from Code.UCBLearner import UCBLearner
from Code.TSLearner import TSLearner
from Code.UCBLearner4 import UCBLearner4
from Code.TSLearner4 import TSLearner4
from Code.UCBLearner5 import UCBLearner5
from Code.TSLearner5 import TSLearner5

%matplotlib inline

In [2]:
cust_json = "customer3.json"
today_customer = 150
var_cust = 10
p_lambda = 1.0
graph_json = "business_full_graph2.json"

arms = settings.prices
environment = Environment(cust_json,today_customer,var_cust,p_lambda, graph_json, arms)
clairvoyant_arms, clairvoyant_reward, exp_c, s= environment.estimate_clairvoyant()
print("\n", clairvoyant_arms, clairvoyant_reward, exp_c)

loading: [                              ] 0.0%
loading: [=                             ] 1.0%
loading: [=                             ] 2.0%
loading: [=                             ] 2.9%
loading: [==                            ] 3.9%
loading: [==                            ] 4.9%
loading: [==                            ] 5.9%
loading: [===                           ] 6.8%
loading: [===                           ] 7.8%
loading: [===                           ] 8.8%
loading: [===                           ] 9.8%
loading: [====                          ] 10.7%
loading: [====                          ] 11.7%
loading: [====                          ] 12.7%
loading: [=====                         ] 13.7%
loading: [=====                         ] 14.6%
loading: [=====                         ] 15.6%
loading: [=====                         ] 16.6%
loading: [======                        ] 17.6%
loading: [======                        ] 18.6%
loading: [======                        ] 19.5%
loa

In [3]:
n_arms = 4
n_products = 5 

feature1 = 0
feature2 = 0

customer = Customer(feature1, feature2)

TIME_HORIZON = 300
NUM_ITERATION = 3

In [4]:
#unknown param

customers = copy.deepcopy(environment.customers)
for customer in customers:
    customer.set_probability_buy(np.ones_like(customer.get_buy_distribution()))
    customer.set_probability_click(np.ones((n_products,n_products)))

UCB

In [ ]:
rewards_iterations = []
for i in range(NUM_ITERATION):
    ucb5_learner = UCBLearner5(n_arms, n_products, customers, graph_json, arms, environment.customers_distribution)
    print('Iteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ucb5_learner.select_superarm()
        report = environment.round(super_arm)
        ucb5_learner.update(super_arm.copy(), report)
    rewards = ucb5_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

Iteration # 1
loading: [                              ] 0.0%
[[0.         0.1        0.         0.         0.1       ]
 [0.65       0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.66666667 0.         0.         0.         0.        ]]
[[0.         0.24137931 0.         0.         0.06896552]
 [0.64       0.         0.         0.         0.        ]
 [0.25       0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.58823529 0.         0.         0.         0.        ]]
[[0.         0.24193548 0.         0.         0.08064516]
 [0.51219512 0.         0.         0.         0.07317073]
 [0.375      0.         0.         0.0625     0.        ]
 [0.6875     0.         0.         0.         0.        ]
 [0.44827586 0.         0.03448276 0.         0.        ]]
[[0.         0.24242424 0.         0.         0.10606061]
 [0.5121

[[0.         0.3025641  0.         0.         0.08205128]
 [0.47461929 0.         0.         0.         0.03553299]
 [0.39910314 0.         0.         0.14349776 0.        ]
 [0.58064516 0.         0.05483871 0.         0.        ]
 [0.43351064 0.         0.05053191 0.         0.        ]]
[[0.         0.2979798  0.         0.         0.08080808]
 [0.4754902  0.         0.         0.         0.03676471]
 [0.40692641 0.         0.         0.13852814 0.        ]
 [0.58359621 0.         0.05993691 0.         0.        ]
 [0.43958869 0.         0.04884319 0.         0.        ]]
loading: [===                           ] 10.0%
[[0.         0.2992     0.         0.         0.0832    ]
 [0.4754902  0.         0.         0.         0.03676471]
 [0.41493776 0.         0.         0.13692946 0.        ]
 [0.58912387 0.         0.05740181 0.         0.        ]
 [0.4345679  0.         0.04691358 0.         0.        ]]
[[0.         0.29780564 0.         0.         0.0830721 ]
 [0.48356808 0.      

[[0.         0.30455259 0.         0.         0.09026688]
 [0.46984925 0.         0.         0.         0.03517588]
 [0.40702479 0.         0.         0.14256198 0.        ]
 [0.58356941 0.         0.04249292 0.         0.        ]
 [0.43799472 0.         0.04485488 0.         0.        ]]
[[0.         0.30662558 0.         0.         0.09090909]
 [0.46863469 0.         0.         0.         0.03444034]
 [0.41082164 0.         0.         0.14428858 0.        ]
 [0.57836338 0.         0.04160888 0.         0.        ]
 [0.43896104 0.         0.04545455 0.         0.        ]]
[[0.         0.30534351 0.         0.         0.09007634]
 [0.47087379 0.         0.         0.         0.03519417]
 [0.41468254 0.         0.         0.14285714 0.        ]
 [0.57493188 0.         0.04223433 0.         0.        ]
 [0.43701799 0.         0.04498715 0.         0.        ]]
[[0.         0.30451128 0.         0.         0.09022556]
 [0.47009569 0.         0.         0.         0.034689  ]
 [0.4145383

[[0.         0.28951487 0.         0.         0.08763693]
 [0.48039216 0.         0.         0.         0.03267974]
 [0.40830946 0.         0.         0.14613181 0.        ]
 [0.58444023 0.         0.0398482  0.         0.        ]
 [0.42735043 0.         0.05213675 0.         0.        ]]
[[0.         0.29072165 0.         0.         0.08814433]
 [0.47868061 0.         0.         0.         0.03298471]
 [0.40651558 0.         0.         0.14730878 0.        ]
 [0.58669166 0.         0.0393627  0.         0.        ]
 [0.42639594 0.         0.05245347 0.         0.        ]]
[[0.         0.29051988 0.         0.         0.08817533]
 [0.47815727 0.         0.         0.         0.03256553]
 [0.4041958  0.         0.         0.14685315 0.        ]
 [0.58534323 0.         0.03896104 0.         0.        ]
 [0.42737896 0.         0.05342237 0.         0.        ]]
[[0.         0.29056795 0.         0.         0.08772819]
 [0.47870662 0.         0.         0.         0.03233438]
 [0.4008264

[[0.         0.28991109 0.         0.         0.09199845]
 [0.484375   0.         0.         0.         0.02944712]
 [0.39525284 0.         0.         0.1496388  0.        ]
 [0.58509142 0.         0.04149086 0.         0.        ]
 [0.43947037 0.         0.05800757 0.         0.        ]]
[[0.         0.28991109 0.         0.         0.09199845]
 [0.4836212  0.         0.         0.         0.02977963]
 [0.39362795 0.         0.         0.14902364 0.        ]
 [0.58589385 0.         0.04120112 0.         0.        ]
 [0.43796992 0.         0.05827068 0.         0.        ]]
[[0.         0.28877825 0.         0.         0.0926848 ]
 [0.48290094 0.         0.         0.         0.03007075]
 [0.39329268 0.         0.         0.14837398 0.        ]
 [0.58558559 0.         0.04296604 0.         0.        ]
 [0.43808339 0.         0.05787181 0.         0.        ]]
[[0.         0.28907436 0.         0.         0.09370258]
 [0.48302108 0.         0.         0.         0.03044496]
 [0.3929648

[[0.         0.29309811 0.         0.         0.09532652]
 [0.47480989 0.         0.         0.         0.0289924 ]
 [0.38707103 0.         0.         0.15083799 0.        ]
 [0.58420762 0.         0.04362231 0.         0.        ]
 [0.42821031 0.         0.05510617 0.         0.        ]]
[[0.         0.29431644 0.         0.         0.09585253]
 [0.47558685 0.         0.         0.         0.02910798]
 [0.38893281 0.         0.         0.15019763 0.        ]
 [0.58497807 0.         0.0433114  0.         0.        ]
 [0.42864296 0.         0.05458187 0.         0.        ]]
[[0.         0.29508697 0.         0.         0.09581935]
 [0.47625698 0.         0.         0.         0.02886406]
 [0.38770686 0.         0.         0.14972419 0.        ]
 [0.58501914 0.         0.043193   0.         0.        ]
 [0.42835895 0.         0.05453644 0.         0.        ]]
[[0.         0.29615035 0.         0.         0.0957866 ]
 [0.4766528  0.         0.         0.         0.02866389]
 [0.3860610

[[0.         0.29198966 0.         0.         0.09431525]
 [0.47714736 0.         0.         0.         0.03073286]
 [0.37940199 0.         0.         0.14285714 0.        ]
 [0.58847737 0.         0.04389575 0.         0.        ]
 [0.42488362 0.         0.05670758 0.         0.        ]]
[[0.         0.29224448 0.         0.         0.09476117]
 [0.47691706 0.         0.         0.         0.03090767]
 [0.37887789 0.         0.         0.14257426 0.        ]
 [0.58900999 0.         0.04359673 0.         0.        ]
 [0.42490534 0.         0.05679428 0.         0.        ]]
loading: [=================             ] 56.7%
[[0.         0.293143   0.         0.         0.09482539]
 [0.47595035 0.         0.         0.         0.03064391]
 [0.37901639 0.         0.         0.14163934 0.        ]
 [0.58871332 0.         0.04379233 0.         0.        ]
 [0.42588727 0.         0.05636743 0.         0.        ]]
[[0.         0.29265823 0.         0.         0.09468354]
 [0.47727273 0.      

[[0.         0.29328309 0.         0.         0.09377078]
 [0.47336683 0.         0.         0.         0.03048576]
 [0.37386364 0.         0.         0.14261364 0.        ]
 [0.57959815 0.         0.04404946 0.         0.        ]
 [0.41508746 0.         0.05721574 0.         0.        ]]
[[0.         0.29252151 0.         0.         0.09353629]
 [0.47419247 0.         0.         0.         0.03030303]
 [0.37365743 0.         0.         0.14245336 0.        ]
 [0.57953239 0.         0.0436949  0.         0.        ]
 [0.41503977 0.         0.05676067 0.         0.        ]]
[[0.         0.29217926 0.         0.         0.09336555]
 [0.47385837 0.         0.         0.         0.03044341]
 [0.37289089 0.         0.         0.14341957 0.        ]
 [0.57924743 0.         0.04484987 0.         0.        ]
 [0.41537909 0.         0.05641394 0.         0.        ]]
[[0.         0.29188596 0.         0.         0.09320175]
 [0.47429136 0.         0.         0.         0.03032301]
 [0.3726150

[[0.         0.29152937 0.         0.         0.09226594]
 [0.4794561  0.         0.         0.         0.02926397]
 [0.37977194 0.         0.         0.14229053 0.        ]
 [0.5792826  0.         0.04358029 0.         0.        ]
 [0.41844197 0.         0.05405405 0.         0.        ]]
[[0.         0.29110512 0.         0.         0.09222179]
 [0.47881107 0.         0.         0.         0.02913479]
 [0.38139764 0.         0.         0.14271654 0.        ]
 [0.57882432 0.         0.04375418 0.         0.        ]
 [0.41993671 0.         0.05411392 0.         0.        ]]
[[0.         0.29072442 0.         0.         0.0925642 ]
 [0.47874524 0.         0.         0.         0.02961009]
 [0.38242514 0.         0.         0.14383898 0.        ]
 [0.57903494 0.         0.04359401 0.         0.        ]
 [0.41976086 0.         0.05412209 0.         0.        ]]
[[0.         0.29045802 0.         0.         0.09236641]
 [0.47842566 0.         0.         0.         0.02944606]
 [0.3826979

[[0.         0.29005808 0.         0.         0.09190297]
 [0.47743966 0.         0.         0.         0.02885624]
 [0.38816362 0.         0.         0.14142733 0.        ]
 [0.57821901 0.         0.04302046 0.         0.        ]
 [0.41865024 0.         0.05572669 0.         0.        ]]
[[0.         0.28996599 0.         0.         0.0914966 ]
 [0.47818134 0.         0.         0.         0.02874314]
 [0.38744589 0.         0.         0.14069264 0.        ]
 [0.57931034 0.         0.04287856 0.         0.        ]
 [0.4180854  0.         0.05581915 0.         0.        ]]
[[0.         0.29043744 0.         0.         0.09155646]
 [0.47815913 0.         0.         0.         0.02886115]
 [0.38740293 0.         0.         0.14020708 0.        ]
 [0.5799701  0.         0.04334828 0.         0.        ]
 [0.41859171 0.         0.05566379 0.         0.        ]]
[[0.         0.29058387 0.         0.         0.09146136]
 [0.47840703 0.         0.         0.         0.02896302]
 [0.3869995

In [ ]:
plt.figure(figsize=(10, 5))
mean = np.mean(rewards_iterations, axis=0)
stdev = np.std(rewards_iterations, axis=0)

plt.plot(range(len(mean)), mean)
plt.fill_between(range(len(mean)), (mean-stdev), (mean+stdev), color='b', alpha=.1)

plt.plot([_ for _ in range(len(rewards))], [clairvoyant_reward for _ in range(len(rewards))])
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
mean_regret = np.mean(regret, axis=0)
std_regret = np.std(regret, axis=0)

plt.plot(range(len(mean_regret)), mean_regret)
plt.fill_between(range(len(mean_regret)), (mean_regret-std_regret), (mean_regret+std_regret), color='b', alpha=.1)
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
cumulative_regret = np.cumsum(regret, axis=1)
cum_reg_mean = np.mean(cumulative_regret, axis=0)
cum_reg_std = np.std(cumulative_regret, axis=0)

plt.plot(range(len(cum_reg_mean)), cum_reg_mean)
plt.fill_between(range(len(cum_reg_mean)), (cum_reg_mean-cum_reg_std), (cum_reg_mean+cum_reg_std), color='b', alpha=.1)
plt.show()

TS

In [ ]:
# set new believed customer for the learner
customers = copy.deepcopy(environment.customers)
for customer in customers:
    customer.set_probability_buy(np.ones_like(customer.get_buy_distribution()))
    customer.set_probability_click(np.ones((n_products,n_products)))

In [ ]:
rewards_iterations = []
for i in range(NUM_ITERATION):
    ts5_learner = TSLearner5(n_arms, n_products, customers, graph_json, arms, environment.customers_distribution)
    print('\nIteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ts5_learner.select_superarm()
        report = environment.round(super_arm)
        ts5_learner.update(super_arm.copy(), report)
    rewards = ts5_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

In [ ]:
plt.figure(figsize=(10, 5))
mean = np.mean(rewards_iterations, axis=0)
stdev = np.std(rewards_iterations, axis=0)

plt.plot(range(len(mean)), mean)
plt.fill_between(range(len(mean)), (mean-stdev), (mean+stdev), color='b', alpha=.1)

plt.plot([_ for _ in range(len(rewards))], [clairvoyant_reward for _ in range(len(rewards))])
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
mean_regret = np.mean(regret, axis=0)
std_regret = np.std(regret, axis=0)

plt.plot(range(len(mean_regret)), mean_regret)
plt.fill_between(range(len(mean_regret)), (mean_regret-std_regret), (mean_regret+std_regret), color='b', alpha=.1)

plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
cumulative_regret = np.cumsum(regret, axis=1)
cum_reg_mean = np.mean(cumulative_regret, axis=0)
cum_reg_std = np.std(cumulative_regret, axis=0)

plt.plot(range(len(cum_reg_mean)), cum_reg_mean)
plt.fill_between(range(len(cum_reg_mean)), (cum_reg_mean-cum_reg_std), (cum_reg_mean+cum_reg_std), color='b', alpha=.1)

plt.show()

In [ ]:
th_rews = list(rew_per_arm.values())
th_rews.sort()
delta_arms = (th_rews[-1] - th_rews[-2]) / today_customer
seen = np.array(ts_learner4.history_interactions)
theoretical_regret = 1024 * np.log(seen)/delta_arms

In [ ]:
plt.plot(range(len(ts_cum_reg_mean)), ts_cum_reg_mean)
plt.fill_between(range(len(ts_cum_reg_mean)), (ts_cum_reg_mean-ts_cum_reg_stdev), (ts_cum_reg_mean+ts_cum_reg_stdev), color='b', alpha=.1)
plt.plot(range(len(theoretical_regret)), theoretical_regret)
plt.show()